## Example using LEAPP

First load the class `LearnPP`

In [9]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris

# we use the iris data set
iris = load_iris()
data = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     columns= ['sl', 'sw', 'pl', 'pw', 'target'])
data['target'] = data['target'].astype("object")

print(data.head())

from leapp.LearnPP import LearnPP

leapp = LearnPP()
# if error during fit, try the following line
# leapp = LearnPP(install_packages=True)

leapp.fit(data)

    sl   sw   pl   pw target
0  5.1  3.5  1.4  0.2      0
1  4.9  3.0  1.4  0.2      0
2  4.7  3.2  1.3  0.2      0
3  4.6  3.1  1.5  0.2      0
4  5.0  3.6  1.4  0.2      0
/home/julien/PycharmProjects/leapp/_tmp.dat


We can not get the pymc3 code. (If you get a `get_node` is `None` error. Try using `LearnPP(install_packages=True)`)

In [10]:
print(leapp.get_pymc_code())

import pymc3 as pm
import theano.tensor as tt
with pm.Model() as model:
    target = pm.Categorical('target', p=[0.3333,0.3333,0.3333])
    pl = pm.Normal('pl', mu=tt.switch(tt.eq(target, 0), 1.462, tt.switch(tt.eq(target, 1), 4.26, 5.552)), sigma=tt.switch(tt.eq(target, 0), 0.1737, tt.switch(tt.eq(target, 1), 0.4699, 0.5519)))
    pw = pm.Normal('pw', mu=tt.switch(tt.eq(target, 0), pl*0.2012+-0.0482, tt.switch(tt.eq(target, 1), pl*0.3311+-0.0843, pl*0.1603+1.136)), sigma=tt.switch(tt.eq(target, 0), 0.1005, tt.switch(tt.eq(target, 1), 0.1234, 0.2627)))
    sw = pm.Normal('sw', mu=tt.switch(tt.eq(target, 0), pw*0.8372+3.2221, tt.switch(tt.eq(target, 1), pw*1.0536+1.3729, pw*0.6314+1.6948)), sigma=tt.switch(tt.eq(target, 0), 0.3725, tt.switch(tt.eq(target, 1), 0.2371, 0.2747)))
    sl = pm.Normal('sl', mu=sw*0.6508+pl*0.7091+pw*-0.5565+1.856, sigma=0.3145)



With this code we can answer queries like give me the class
for the given `pl` and `sw` or give `sl` given the class and `pl`.

In [12]:
# query 1: p(class|pl=1.4, sw=3.6)
import pymc3 as pm
import theano.tensor as tt

# define the model and set the observed variables with values
with pm.Model() as model:
    target = pm.Categorical('target', p=[0.3333,0.3333,0.3333])
    pl = pm.Normal('pl', observed=1.4, mu=tt.switch(tt.eq(target, 0), 1.462, tt.switch(tt.eq(target, 1), 4.26, 5.552)), sigma=tt.switch(tt.eq(target, 0), 0.1737, tt.switch(tt.eq(target, 1), 0.4699, 0.5519)))
    pw = pm.Normal('pw', mu=tt.switch(tt.eq(target, 0), pl*0.2012+-0.0482, tt.switch(tt.eq(target, 1), pl*0.3311+-0.0843, pl*0.1603+1.136)), sigma=tt.switch(tt.eq(target, 0), 0.1005, tt.switch(tt.eq(target, 1), 0.1234, 0.2627)))
    sw = pm.Normal('sw', observed=3.6, mu=tt.switch(tt.eq(target, 0), pw*0.8372+3.2221, tt.switch(tt.eq(target, 1), pw*1.0536+1.3729, pw*0.6314+1.6948)), sigma=tt.switch(tt.eq(target, 0), 0.3725, tt.switch(tt.eq(target, 1), 0.2371, 0.2747)))
    sl = pm.Normal('sl', mu=sw*0.6508+pl*0.7091+pw*-0.5565+1.856, sigma=0.3145)

# draw samples from this model
with model:
    samples = pm.sample(draws=1000, tune=10)

# what class was observed the most?
print([i for i, j in zip(np.unique(samples['target']), np.bincount(samples['target']))][0])

Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>CategoricalGibbsMetropolis: [target]
>NUTS: [sl, pw]
Sampling 4 chains, 0 divergences: 100%|██████████| 4040/4040 [00:04<00:00, 935.08draws/s] 
/home/julien/anaconda3/envs/leapp/lib/python3.7/site-packages/xarray/core/nputils.py:227: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)
The acceptance probability does not match the target. It is 0.9571670402356157, but should be close to 0.8. Try to increase the number of tuning steps.
The acceptance probability does not match the target. It is 0.9383869964614966, but should be close to 0.8. Try to increase the number of tuning steps.
The acceptance probability does not match the target. It is 0.9723978455049981, but should be close to 0.8. Try to increase the number of tuning steps.
The acceptance probability does not match the target. It is 0.9522782196754975, but should be close to 0.8. Try to increase the number of tuning st

[0]


In [13]:
# query 2: most probable sl given the class 1 and pl=1.5

# we define the following helper function
def mep(samples, bins=1000):
    # get bins and boarders
    b, m = np.histogram(samples, bins=bins)
    # get the boarders for the bin
    d, c = (m[[k for k, (i, j) in enumerate(zip(b, m)) if i == max(b)][0] - 1], m[[k for k, (i, j) in enumerate(zip(b, m)) if i == max(b)][0]])
    # get the samples inside of this bin
    s1 = sorted(samples)
    s2 = samples[samples >= d]
    s3 = s2[s2 <= c]
    # return the mean of this bin
    return np.mean(s3)

# define the model and set the observed variables with values
with pm.Model() as model:
    target = pm.Categorical('target', observed=1, p=[0.3333,0.3333,0.3333])
    pl = pm.Normal('pl', observed=1.5, mu=tt.switch(tt.eq(target, 0), 1.462, tt.switch(tt.eq(target, 1), 4.26, 5.552)), sigma=tt.switch(tt.eq(target, 0), 0.1737, tt.switch(tt.eq(target, 1), 0.4699, 0.5519)))
    pw = pm.Normal('pw', mu=tt.switch(tt.eq(target, 0), pl*0.2012+-0.0482, tt.switch(tt.eq(target, 1), pl*0.3311+-0.0843, pl*0.1603+1.136)), sigma=tt.switch(tt.eq(target, 0), 0.1005, tt.switch(tt.eq(target, 1), 0.1234, 0.2627)))
    sw = pm.Normal('sw', mu=tt.switch(tt.eq(target, 0), pw*0.8372+3.2221, tt.switch(tt.eq(target, 1), pw*1.0536+1.3729, pw*0.6314+1.6948)), sigma=tt.switch(tt.eq(target, 0), 0.3725, tt.switch(tt.eq(target, 1), 0.2371, 0.2747)))
    sl = pm.Normal('sl', mu=sw*0.6508+pl*0.7091+pw*-0.5565+1.856, sigma=0.3145)

# draw samples from this model
with model:
    samples = pm.sample(draws=1000, tune=10)

# what class was observed the most?
print(mep(samples['sl']))

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sl, sw, pw]
Sampling 4 chains, 0 divergences: 100%|██████████| 4040/4040 [00:04<00:00, 817.15draws/s]
The acceptance probability does not match the target. It is 0.9331436202405704, but should be close to 0.8. Try to increase the number of tuning steps.
The acceptance probability does not match the target. It is 0.9442912081621049, but should be close to 0.8. Try to increase the number of tuning steps.
The acceptance probability does not match the target. It is 0.96756551447661, but should be close to 0.8. Try to increase the number of tuning steps.
The acceptance probability does not match the target. It is 0.9456371111710484, but should be close to 0.8. Try to increase the number of tuning steps.


3.9238394538837866
